In [18]:
%matplotlib inline
import re
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from elasticsearch import Elasticsearch, exceptions as es_exceptions
from elasticsearch.helpers import scan
es = Elasticsearch(hosts=[{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

# To list all indices, do 
# curl -XGET 'http://atlas-kibana.mwt2.org:9200/_settings?pretty=1' 
ind="jobs_archive_2016-09*"

# one can also use the following command to see the return json data.
# curl -XGET 'http://atlas-kibana.mwt2.org:9200/jobs_archive_2016*/_search?pretty=1&size=1'

x = {
    "fields": ["corecount","modificationtime","jobstatus",
               "piloterrorcode","exeerrorcode","transerrorcode"],
    "query": {
        "bool": {
            "must_not": [{ "wildcard" : { "computingsite" : "ANALY_*" }}],
            "must" : [ { "range" : { "maxattempt" : { "lte" : 5 }}}]
        }
    }
}

nfinished = []
ntotal = []
timerange0 = datetime.strptime("2016-09-01","%Y-%m-%d")
timerange1 = datetime.strptime("2016-10-01","%Y-%m-%d")

# timerange1-timerange0 is a timedelta object
ndays = (timerange1 - timerange0).days  
for i in range(0, ndays):
    nfinished.append(0)
    ntotal.append(0)

page = es.search(index=ind, body=x, search_type="scan", scroll="1m")
scroll_size = page['hits']['total']

db=0
while (scroll_size > 0):
    try:
        sid = page['_scroll_id']
        page = es.scroll(scroll_id = sid, scroll = '1m')
        scroll_size = len(page['hits']['hits'])
        for job in page['hits']['hits']:
            timestamp = datetime.strptime(job['fields']['modificationtime'][0],"%Y-%m-%dT%H:%M:%S")
            if (timestamp<timerange0 or timestamp>timerange1):
                continue 
            piloterrorcode = job['fields']['piloterrorcode'][0]
            exeerrorcode = job['fields']['exeerrorcode'][0]
            
            i_day = (timestamp-timerange0).days
            jobstatus = job['fields']['jobstatus'][0]
            ntotal[i_day] += 1
            if (db == 0):
                db = 1
                
            if (jobstatus.find("finished") == 0):
                nfinished[i_day] += 1
                print("%s %d %d" % (jobstatus, piloterrorcode, exeerrorcode))
            break
    except:
        continue

print(nfinished)
print(ntotal)



[560690, 382662, 245713, 279860, 221175, 310246, 437435, 501813, 426546, 516326, 447263, 369147, 396011, 455499, 494968, 419121, 409432, 357266, 61340, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[689798, 461344, 290382, 339503, 300727, 407221, 549432, 644648, 564526, 706967, 612571, 596346, 583024, 734222, 810476, 573456, 568776, 485687, 90045, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [26]:
for i in range(0, ndays):
    try:
        print("[%d] %f" % (i, float(nfinished[i])/ntotal[i]))
    except:
        continue

[0] 0.812832
[1] 0.829450
[2] 0.846172
[3] 0.824323
[4] 0.735468
[5] 0.761861
[6] 0.796159
[7] 0.778429
[8] 0.755583
[9] 0.730340
[10] 0.730141
[11] 0.619015
[12] 0.679236
[13] 0.620383
[14] 0.610713
[15] 0.730869
[16] 0.719848
[17] 0.735589
[18] 0.681215
